## Apply the bortezomib signature

We previously identified a set of morphology features (in `1.build-signature.ipynb`) that are most significantly different between resistant and sensitive clones.
We used only the training set samples to identify this signature (the set of 5 WT and 5 resistant clones).

Here, we apply this signature to the full analytical data set.

### Singscore

We use singscore to apply the signatures to the data.

> Foroutan, M., Bhuva, D.D., Lyu, R. et al. Single sample scoring of molecular phenotypes. BMC Bioinformatics 19, 404 (2018). https://doi.org/10.1186/s12859-018-2435-4

In singscore, we use the features that are significantly higher in resistant samples, and features that are significantly lower in resistant samples to generate an output score.

A high score represents a sample with a morphology profile that is concordant with the signature.
The per-sample score can be interpreted as the mean percentile rank of the target features.

#### Random permutations

We also calculate singscore with several random permutations.
We randomly shuffle the feature rankings to derive a range in which a sample may be scored simply by chance.

In [1]:
suppressPackageStartupMessages(library(dplyr))
suppressPackageStartupMessages(library(singscore))

source(file.path("utils", "singscore_utils.R"))

In [2]:
seed <- 1234
num_permutations <- 1000
dataset <- "bortezomib"

data_dir <- "data"
input_results_dir <- file.path("results", "signatures")
output_dir <- file.path("results", "singscore")

data_file <- file.path(data_dir, paste0(dataset, "_signature_analytical_set.tsv.gz"))
feat_file <- file.path(data_dir, "dataset_features_selected.tsv")
signature_file <- file.path(input_results_dir, paste0("signature_summary_", dataset, "_signature.tsv.gz"))
tukey_file <- file.path(input_results_dir, paste0("tukey_results_", dataset, "_signature.tsv.gz"))
output_results_file <- file.path(output_dir, paste0("singscore_results", dataset, ".tsv.gz"))

In [3]:
set.seed(seed)

In [4]:
# Load feature selected features
all_selected_features_df <- readr::read_tsv(feat_file, col_types = readr::cols())
head(all_selected_features_df, 3)

features                     dataset   
1 Cells_AreaShape_Compactness  bortezomib
2 Cells_AreaShape_Eccentricity bortezomib
3 Cells_AreaShape_Extent       bortezomib

In [5]:
# Load profiles
bulk_col_types <- readr::cols(
    .default = readr::col_double(),
    Metadata_Plate = readr::col_character(),
    Metadata_Well = readr::col_character(),
    Metadata_cell_count = readr::col_integer(),
    Metadata_batch = readr::col_character(),
    Metadata_clone_number = readr::col_character(),
    Metadata_plate_map_name = readr::col_character(),
    Metadata_treatment = readr::col_character(),
    Metadata_dataset = readr::col_character(),
    Metadata_clone_type = readr::col_character(),
    Metadata_clone_type_indicator = readr::col_character(),
    Metadata_model_split = readr::col_character(),
    Metadata_cell_density = readr::col_character(),
    Metadata_treatment_time = readr::col_character(),
    Metadata_unique_sample_name = readr::col_character(),
    Metadata_time_to_adhere = readr::col_character()
)

data_df <- readr::read_tsv(data_file, col_types = bulk_col_types)

# Apply feature selection performed in 0.compile-bulk-datasets
selected_features <- all_selected_features_df %>%
    dplyr::filter(dataset == !!dataset) %>%
    dplyr::pull(features)

data_df <- data_df %>%
    dplyr::select(starts_with("Metadata"), all_of(selected_features))

print(dim(data_df))
head(data_df, 4)

[1] 525 800


Metadata_Plate Metadata_Well Metadata_batch     Metadata_cell_count
1 219907         B02           2021_03_03_Batch12 6139               
2 219907         B03           2021_03_03_Batch12 4567               
3 219907         B04           2021_03_03_Batch12 5624               
4 219907         B05           2021_03_03_Batch12 5894               
  Metadata_cell_density Metadata_celltype_shorthand_from_plate_graph
1 2.5x10^3 cells/well   1                                           
2 2.5x10^3 cells/well   2                                           
3 2.5x10^3 cells/well   3                                           
4 2.5x10^3 cells/well   4                                           
  Metadata_clone_number Metadata_date Metadata_plate_map_name
1 WT_parental           20210205      219814                 
2 CloneA                20210205      219814                 
3 CloneE                20210205      219814                 
4 WT clone 01           20210205      219814                 
  Metadata_time_to_adhere ⋯ Nuclei_Texture_InfoMeas2_AGP_5_02
1 48 hr                   ⋯ -0.6413393                       
2 48 hr                   ⋯ -0.5787319                       
3 48 hr                   ⋯ -1.0043336                       
4 48 hr                   ⋯ -1.3103629                       
  Nuclei_Texture_InfoMeas2_DNA_5_02 Nuclei_Texture_InfoMeas2_ER_20_01
1 -0.8357316                         0.03365632                      
2 -0.5453454                        -0.94021703                      
3 -1.1624983                        -1.46650377                      
4 -1.2091500                        -1.24142807                      
  Nuclei_Texture_InfoMeas2_ER_5_00 Nuclei_Texture_InfoMeas2_RNA_5_00
1 -0.3288047                       -0.5638985                       
2 -0.7268092                       -0.5434135                       
3 -1.5142510                       -1.1387449                       
4 -0.8951256                       -1.1444221                       
  Nuclei_Texture_InverseDifferenceMoment_ER_20_01
1 0.00908339                                     
2 0.85805603                                     
3 1.58735084                                     
4 1.48149452                                     
  Nuclei_Texture_InverseDifferenceMoment_RNA_20_01
1 0.3270510                                       
2 0.9887777                                       
3 1.2585914                                       
4 2.0026168                                       
  Nuclei_Texture_SumAverage_DNA_20_03 Nuclei_Texture_SumEntropy_DNA_20_03
1 -0.4452347                          -1.016692                          
2 -1.2714808                          -0.782711                          
3 -0.9945962                          -1.099421                          
4 -0.6836113                          -1.067353                          
  Nuclei_Texture_SumVariance_RNA_20_03
1 -0.7072398                          
2 -0.7730713                          
3 -0.9592103                          
4 -1.0625873

In [6]:
# Load signatures
sig_col_types <- readr::cols(
    features = readr::col_character(),
    non_specific_exclude = readr::col_logical(),
    final_signature = readr::col_logical(),
    dataset = readr::col_character()
)

signature_df <- readr::read_tsv(signature_file, col_types = sig_col_types)

print(dim(signature_df))
head(signature_df, 4)

[1] 782   8


features                     non_status_significant_exclude batch_exclude
1 Cells_AreaShape_Compactness  FALSE                          FALSE        
2 Cells_AreaShape_Eccentricity  TRUE                          FALSE        
3 Cells_AreaShape_Extent       FALSE                          FALSE        
4 Cells_AreaShape_FormFactor   FALSE                          FALSE        
  cell_count_exclude non_specific_exclude treatment_time_exclude
1 FALSE              TRUE                 FALSE                 
2  TRUE              TRUE                 FALSE                 
3 FALSE              TRUE                 FALSE                 
4  TRUE              TRUE                 FALSE                 
  final_signature dataset   
1 FALSE           bortezomib
2 FALSE           bortezomib
3 FALSE           bortezomib
4 FALSE           bortezomib

In [7]:
# Load Tukey results (to determine if feature is "up" or "down")
tukey_cols <- readr::cols(
    term = readr::col_character(),
    comparison = readr::col_character(),
    estimate = readr::col_double(),
    conf.low = readr::col_double(),
    conf.high = readr::col_double(),
    adj.p.value = readr::col_double(),
    feature = readr::col_character(),
    neg_log_adj_p = readr::col_double(),
    dataset = readr::col_character()
)

tukey_df <- readr::read_tsv(tukey_file, col_types = tukey_cols)

print(dim(tukey_df))
head(tukey_df, 4)

[1] 48484     9


term                          comparison                           
1 Metadata_clone_type_indicator 1-0                                  
2 Metadata_treatment_time       4 hr-13 hr                           
3 Metadata_batch                2021_03_03_Batch13-2021_03_03_Batch12
4 Metadata_batch                2021_03_03_Batch15-2021_03_03_Batch12
  estimate    conf.low    conf.high adj.p.value
1  0.78076946  0.66152296 0.9000160 0.0000000  
2  0.05503349 -0.07099084 0.1810578 0.3901031  
3 -0.02082200 -0.32241818 0.2807742 0.9999567  
4 -0.15303419 -0.45233883 0.1462704 0.6824337  
  feature                                     neg_log_adj_p dataset   
1 Nuclei_RadialDistribution_FracAtD_Mito_3of4          Inf  bortezomib
2 Nuclei_RadialDistribution_FracAtD_Mito_3of4 4.088207e-01  bortezomib
3 Nuclei_RadialDistribution_FracAtD_Mito_3of4 1.881967e-05  bortezomib
4 Nuclei_RadialDistribution_FracAtD_Mito_3of4 1.659395e-01  bortezomib

In [8]:
# Subset data to process dataset-specific signature
signature_subset_df <- signature_df %>%
    dplyr::filter(dataset == !!dataset, final_signature)

tukey_subset_df <- tukey_df %>%
    dplyr::filter(
        dataset == !!dataset,
        term == "Metadata_clone_type_indicator",
        feature %in% signature_subset_df$features
    ) %>%
    dplyr::arrange(desc(estimate))

# Ensure that the comparison is always resistant vs. senstive
# and never the other way around!
stopifnot(length(table(tukey_subset_df$comparison)) == 1)

# Determine feature direction
up_features <- tukey_subset_df %>% dplyr::filter(estimate > 0) %>% dplyr::pull(feature)
down_features <- tukey_subset_df %>% dplyr::filter(estimate < 0) %>% dplyr::pull(feature)

# Store signature for downstream analyses
signature_features <- list("up" = up_features, "down" = down_features)

In [9]:
signature_features

$up
 [1] "Cytoplasm_RadialDistribution_MeanFrac_Mito_3of4"
 [2] "Nuclei_RadialDistribution_RadialCV_ER_1of4"     
 [3] "Nuclei_RadialDistribution_MeanFrac_RNA_1of4"    
 [4] "Cells_RadialDistribution_RadialCV_ER_2of4"      
 [5] "Nuclei_RadialDistribution_MeanFrac_AGP_1of4"    
 [6] "Cells_RadialDistribution_RadialCV_RNA_2of4"     
 [7] "Nuclei_RadialDistribution_MeanFrac_AGP_2of4"    
 [8] "Cells_RadialDistribution_RadialCV_Mito_1of4"    
 [9] "Cytoplasm_Correlation_K_DNA_AGP"                
[10] "Cells_Correlation_K_DNA_AGP"                    
[11] "Nuclei_RadialDistribution_RadialCV_RNA_2of4"    
[12] "Cytoplasm_Granularity_2_AGP"                    
[13] "Nuclei_AreaShape_Zernike_6_0"                   
[14] "Nuclei_Granularity_7_DNA"                       

$down
 [1] "Cytoplasm_Correlation_Manders_RNA_AGP"          
 [2] "Cytoplasm_Intensity_MassDisplacement_ER"        
 [3] "Cells_AreaShape_Zernike_4_2"                    
 [4] "Nuclei_Texture_InfoMeas1_RNA_5_00"              
 [5] "Cells_Texture_InfoMeas2_DNA_5_02"               
 [6] "Cytoplasm_Texture_InfoMeas2_Mito_5_01"          
 [7] "Cytoplasm_Intensity_IntegratedIntensityEdge_RNA"
 [8] "Cytoplasm_Correlation_K_AGP_DNA"                
 [9] "Cells_Texture_Correlation_Mito_10_00"           
[10] "Cytoplasm_Texture_AngularSecondMoment_Mito_5_02"
[11] "Cytoplasm_Correlation_Manders_ER_AGP"           
[12] "Nuclei_Texture_Correlation_RNA_10_03"           
[13] "Nuclei_Texture_Correlation_RNA_10_01"           
[14] "Nuclei_Texture_Correlation_Mito_10_02"          
[15] "Cells_Correlation_Manders_Mito_RNA"             
[16] "Cytoplasm_Texture_InfoMeas2_Mito_5_00"          
[17] "Nuclei_AreaShape_Zernike_9_3"                   
[18] "Cells_Texture_Correlation_Mito_10_02"           
[19] "Cytoplasm_Correlation_Manders_ER_RNA"           
[20] "Nuclei_Texture_Correlation_Mito_10_00"          
[21] "Nuclei_Texture_Correlation_Mito_10_03"          
[22] "Nuclei_Texture_Correlation_DNA_5_00"            
[23] "Cytoplasm_Intensity_MassDisplacement_Mito"      
[24] "Cytoplasm_Correlation_RWC_Mito_ER"              
[25] "Cells_Correlation_Manders_Mito_ER"              
[26] "Nuclei_Texture_Correlation_Mito_10_01"          
[27] "Nuclei_Texture_Correlation_DNA_10_00"           
[28] "Nuclei_Texture_Correlation_DNA_10_02"           
[29] "Cytoplasm_Correlation_RWC_DNA_ER"               
[30] "Cells_Intensity_IntegratedIntensity_DNA"        
[31] "Cytoplasm_Correlation_RWC_DNA_RNA"

In [10]:
singscore_output = singscorePipeline(
    df = data_df,
    sig_feature_list = signature_features,
    num_permutations = num_permutations
)

full_results_df <- singscore_output[["results"]]
permuted <- singscore_output[["permuted"]]

# Get max and minimum values of permutation results
min_val <- quantile(as.vector(as.matrix(permuted)), 0.05)
max_val <- quantile(as.vector(as.matrix(permuted)), 0.95)

# Annotate some key metadata and store to list
sing_score_results_df <- full_results_df %>%
    dplyr::mutate(
        dataset = dataset,
        min_permuted_value = min_val,
        max_permuted_value = max_val
    )

Warning message:
“'tidy.numeric' is deprecated.
See help("Deprecated")”Warning message:
“`data_frame()` is deprecated as of tibble 1.1.0.
Please use `tibble()` instead.
This warning is displayed once every 8 hours.
Call `lifecycle::last_warnings()` to see where this warning was generated.”

In [11]:
sing_score_results_df %>% readr::write_tsv(output_results_file)

print(dim(sing_score_results_df))
head(sing_score_results_df)

[1] 525  28


Metadata_Plate Metadata_Well Metadata_batch     Metadata_cell_count
1 219907         B02           2021_03_03_Batch12 6139               
2 219907         B03           2021_03_03_Batch12 4567               
3 219907         B04           2021_03_03_Batch12 5624               
4 219907         B05           2021_03_03_Batch12 5894               
5 219907         B06           2021_03_03_Batch12 1277               
6 219907         B07           2021_03_03_Batch12 5452               
  Metadata_cell_density Metadata_celltype_shorthand_from_plate_graph
1 2.5x10^3 cells/well   1                                           
2 2.5x10^3 cells/well   2                                           
3 2.5x10^3 cells/well   3                                           
4 2.5x10^3 cells/well   4                                           
5 2.5x10^3 cells/well   5                                           
6 2.5x10^3 cells/well   1                                           
  Metadata_clone_number Metadata_date Metadata_plate_map_name
1 WT_parental           20210205      219814                 
2 CloneA                20210205      219814                 
3 CloneE                20210205      219814                 
4 WT clone 01           20210205      219814                 
5 WT clone 02           20210205      219814                 
6 WT_parental           20210205      219814                 
  Metadata_time_to_adhere ⋯ TotalScore  TotalDispersion UpScore     
1 48 hr                   ⋯ -0.15194809 385.4760        -0.155319940
2 48 hr                   ⋯ -0.03125574 519.6513        -0.089006696
3 48 hr                   ⋯  0.00999497 471.4668        -0.101748512
4 48 hr                   ⋯ -0.06232360 274.2810        -0.080729167
5 48 hr                   ⋯ -0.17249525 398.0781        -0.009207589
6 48 hr                   ⋯ -0.18551606 391.4064        -0.155598958
  UpDispersion DownScore    DownDispersion Metadata_permuted_p_value dataset   
1 185.3250      0.003371848 200.1510       0.941                     bortezomib
2 350.6349      0.057750956 169.0164       0.584                     bortezomib
3 284.6592      0.111743482 186.8076       0.430                     bortezomib
4  99.3342      0.018405567 174.9468       0.720                     bortezomib
5 168.2751     -0.163287659 229.8030       0.968                     bortezomib
6 111.1950     -0.029917100 280.2114       0.975                     bortezomib
  min_permuted_value max_permuted_value
1 -0.1548019         0.1589312         
2 -0.1548019         0.1589312         
3 -0.1548019         0.1589312         
4 -0.1548019         0.1589312         
5 -0.1548019         0.1589312         
6 -0.1548019         0.1589312